### Data Transform

Analysis Process:

* Perform EDA and check what are all the datapoints we have to upload. As in, do we have to upload patient, specimen and so on?

* Within the datapoints what are the distinct values for each attribute (e.g. Say specimen has bodysite column, then what are the distinct values of that column?)

* Map the values directly to the values that exist in Case Browser Data Dictionary

* If not, then we need to apply certain transformations which come at a later stage

In [1]:
import os
import yaml
import logging
import pandas as pd
from datetime import datetime
from modules.transform_dst_mod import transform_dst
from modules.excel_mod import preprocess_data_from_excel
from modules.transform_biochem_mod import transform_biochem
from modules.transform_culture_mod import transform_culture
from modules.transform_specimen_mod import transform_specimen
from modules.transform_microscopy_mod import transform_microscopy
from modules.transform_patient_case_mod import transform_condition
from modules.transform_patient_case_mod import  transform_patient_case
from modules.transform_Georgia_mod import create_patient_condition_table
from modules.transform_regimen_treatment_mod import transform_regimen_treatment, transform_regimen

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

# Configuring the logging settings
logging.basicConfig(filename='../logs/Data_main.log'.format(datetime.now().strftime("%Y-%m-%d")), level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

#### Helper Functions

These functions are to be used in special cases where small transformations are required.

In [2]:
# Drop the first row if needed. Sometimes this row is an example that we don't need.
def drop_example_row(dataframe, rows_to_drop):
    dataframe = dataframe.drop(index=rows_to_drop).reset_index(drop=True)
    return dataframe

# If the height column is in meters, it needs to be converted to centimeters. 
# I noticed that for file Case Browser Bulk upload TBHULID 3 FINAL.xlsx the weight and-
# height values were swapped. So i had to manually fix that.
def swap_values(row):
    weight = row['weight']
    height = row['height']
    
    if 'm' in weight and 'kg' in height:
        row['weight'], row['height'] = height, weight
    elif 'kg' in weight and 'm' in height:
        row['weight'], row['height'] = weight, height
    
    return row

# Function that checks for duplicates in a dataframe
def check_dups(dataframe, col_name):
    if dataframe.duplicated(subset=col_name).sum() == 0:
        print(f'No duplicates in {col_name}')
    else:
        initial_dups = dataframe.duplicated(subset=col_name).sum()
        print(f'Duplicates in {col_name}: {initial_dups}')
        
        # Call the drop_dups function to drop the duplicates
        drop_dups(dataframe, col_name)
        print(f'Dropped duplicates in {col_name}. Duplicate count shoul be zero: {dataframe.duplicated(subset=col_name).sum()}')

def drop_dups(dataframe, col_name):
    dataframe.drop_duplicates(subset=col_name, inplace=True)
    dataframe.reset_index(drop=True, inplace=True)

In [ ]:
# The path to the xlsx files.
with open('../config/config.yml', 'r') as config_file:
    config = yaml.safe_load(config_file)

# Setting the path for the excel file found in the config file.
file_path = config["MLI_dec_2023_file_path"]
file_paths = [file_path]

In [ ]:
# Extract the sheets of interest from the excel file and into csv form
# data_folder = '../COUNTRIES/USA/data/interm/'
# country_month = 'MLI_dec_2023'
# preprocess_data_from_excel(file_paths, data_folder, country_month)

In [ ]:
# Set the dataframes after they have been extracted from the excel file.
patient_case = pd.read_csv('../COUNTRIES/USA/data/interm/MLI_dec_2023_1_20240129.csv')
microscopy = pd.read_csv('../COUNTRIES/USA/data/interm/MLI_dec_2023_2_20240129.csv')
culture = pd.read_csv('../COUNTRIES/USA/data/interm/MLI_dec_2023_3_20240129.csv')
specimen = pd.read_csv('../COUNTRIES/USA/data/interm/MLI_dec_2023_4_20240129.csv')
# dst = pd.read_csv('Mali/data/interm/Mali_jan_3_dst.csv')
# regimen_treatment = pd.read_csv('Mexico/data/interm/Mexico_nov_6.csv')
# biochem = pd.read_csv('Mexico/data/interm/Mexico_nov_7.csv')

#### In cases where the country data does not have a patient case file, use this to create the registrationdate column.

In [ ]:
# CaseBrowser_prod_data = pd.read_csv('../countries/Mali/data/raw/Mali_Prod_data_01_16_24.csv')
# registrationdate = CaseBrowser_prod_data[['patient_local_identifier', 'registrationdate']]

# Changing the column names to match the target dataframe
# registrationdate.rename(columns={'patient_local_identifier':'identifier'}, inplace=True)

In [ ]:
# patient_case = pd.read_csv('Mexico/data/interm/Mexico_nov_1.csv')
# dst = pd.read_csv('Mexico/data/interm/Mexico_nov_5.csv')

#### Patient Case dataframe:

For the patient table, refer to the first three columns of the 'Patient Case' tab. 

For condition, please refer to the remaining columns. 

There is a particular format as to how the condition table is loaded. Dataentry_Pune or TBDM is good table to refer. 

I believe column comorbidity is enclosed within {} where we aggregate the multiple values.

In [ ]:
patient_case.head(20)

In [ ]:
# Some of the column names have spaces in them. I stripped the spaces from the column names.
patient_case.columns = patient_case.columns.str.replace(' ', '')

In [ ]:
patient_case['registrationdate'].unique()

In [ ]:
patient_case.query('registrationdate == "29-Feb-2021"')

In [ ]:
# Change the value 29-Feb-2021 to 2021-02-28. There is no 29th of February in 2021.
patient_case['registrationdate'] = patient_case['registrationdate'].replace('29-Feb-2021', '2021-02-28')

In [ ]:
# Drop the first row if needed. Sometimes this row is an example 
# rows_to_drop = [0]
# patient_case = drop_example_row(patient_case, rows_to_drop)

# Drop the last column in cases where there was a comment written in the excel file
# patient_case = patient_case.iloc[:, :-1]

In [ ]:
new_patient_case = transform_patient_case(patient_case)

In [ ]:
new_patient_case.head(15)

In [ ]:
new_patient_case['weight'] = new_patient_case['weight'].replace('-', pd.NA)

In [ ]:
# Registration dates:
registration = new_patient_case[['identifier', 'registrationdate']]

# Patient table:
patient = new_patient_case[['identifier', 'gender', 'managingorganizationid']]

new_order_patient = ['gender', 'managingorganizationid', 'identifier']

patient = patient.reindex(columns=new_order_patient)

In [ ]:
patient.head()

#### Condition dataframe:

In [ ]:
col_exclude = ['gender', 'managingorganizationid']
condition = new_patient_case.drop(columns=col_exclude)

In [ ]:
condition.head(20)

In [ ]:
# I noticed that for some files the weight and height values were swapped. So i had to manually fix that.
# condition = condition.apply(swap_values, axis=1)

In [ ]:
new_condition = transform_condition(condition)

In [ ]:
new_condition.head(20)

In [ ]:
# The ageonset column needs to be a string type.
new_condition['ageonset'] = new_condition['ageonset'].astype(str)

#### Microscopy dataframe:

In [ ]:
microscopy.head(15)

In [ ]:
microscopy['result'].unique()

In [ ]:
# rows_to_drop = [0]
# microscopy = drop_example_row(microscopy, rows_to_drop)

In [ ]:
# microscopy = microscopy.iloc[:, :-2]

In [ ]:
# Change the value 29/02/2021 to 2021-02-28. There is no 29th of February in 2021.
microscopy['collected_date'] = microscopy['collected_date'].replace('29/02/2021', '2021-02-28')

In [ ]:
new_microscopy = transform_microscopy(microscopy, registration)

In [ ]:
new_microscopy.head(40)

#### Culture dataframe:

In [ ]:
culture.head(20)

In [ ]:
culture['collected_date'].unique()

In [ ]:
# culture = drop_example_row(culture, rows_to_drop)

In [ ]:
# culture = culture.iloc[:, :-1]

In [ ]:
culture['collected_date'] = culture['collected_date'].replace('3/12/2021', '2021-03-12')

In [ ]:
new_culture = transform_culture(culture, registration)

In [ ]:
new_culture.head(20)

In [ ]:
# For all the identifiers where the isued column is NaN, replace the NaN with the registration date for that identifier.
# new_culture['issued'] = new_culture['issued'].fillna(new_culture['identifier'].map(registration.set_index('identifier')['registrationdate']))

#### Specimen dataframe:

In [ ]:
specimen.head(20)

In [ ]:
specimen['collected_date'].unique()

In [ ]:
specimen['collected_date'] = specimen['collected_date'].replace('29/02/2021', '2021-02-28')

In [ ]:
# specimen = drop_example_row(specimen, rows_to_drop)

In [ ]:
# specimen = specimen.iloc[:, :-1]

In [ ]:
new_specimen = transform_specimen(specimen, registration)

In [ ]:
new_specimen.head(40)

#### DST dataframe:
How can I turn the drug columns into a JSON format that looks like this: 
* {({code:H},S),({code:R},S),({code:E},S),({code:S},S)}

Urvi said: 
* The way she usually goes around this is that she has two columns, one for drug code and another for dst results(R,S,I). 
* Then she concats those in the format ({code:H},S). 
* Now for each patient, specimen, dst test combo, she will have multiple rows based on drug:result combo. 
* In the above example there will be a row for ({code:H},S), then a row for ({code:R},S), then a row for ({code:E},S) and ({code:S},S). 
* Finally I will use string_agg on the conacat column and group  by patient, specimen, dst test

In [ ]:
dst.head(40)

In [ ]:
rows_to_drop = [0, 1, 2, 3]
dst = drop_example_row(dst, rows_to_drop)

In [ ]:
# dst['specimen'] = dst['specimen'].str.replace('_S_1', '')

# # I need to change some value in the specimen column to match the values in the specimen dataframe.
# dst['specimen'] = dst['specimen'].replace('MAL09-2040-3', 'MAL092040-3')
# dst['specimen'] = dst['specimen'].replace('MAL09-2040-2', 'MAL092040-2')

# dst['specimen local'] = dst['specimen local'].replace('MAL09-2040-3', 'MAL092040-3_S_3')
# dst['specimen local'] = dst['specimen local'].replace('MAL09-2040-2', 'MAL092040-2_S_2')

In [ ]:
new_dst = transform_dst(dst, registration)

In [ ]:
new_dst.head(50)

In [ ]:
# Keep the indecies that are valid
indexes_to_keep = [9, 11, 13, 20]
new_dst = new_dst.loc[indexes_to_keep]
new_dst.reset_index(drop=True, inplace=True)

#### Regimen and Treatment dataframe:

In [ ]:
regimen_treatment.head(40)

In [ ]:
rows_to_drop = [0, 1]
regimen_treatment = drop_example_row(regimen_treatment, rows_to_drop)

In [ ]:
# The value 'ND' in the end date column for Case Browser Bulk upload TBHULID 3 FINAL.xlsx-
# is changed to NaN
# regimen_treatment['end date'] = regimen_treatment['end date'].replace('ND', np.nan)

In [ ]:
new_regimen_treatment = transform_regimen_treatment(regimen_treatment)

In [ ]:
new_regimen_treatment.head(40)

In [ ]:
# Now that everything is cleaned up, I will split the dataframe to make the regimen and treatment dataframes
treatment_cols = ['identifier', 'start', 'end', 'outcome']
treatment = new_regimen_treatment[treatment_cols]

In [ ]:
treatment = pd.merge(treatment, registration, on='identifier')

# * Re-order the columns to match how they look in the database.
new_order = ['identifier', 'registrationdate', 'start', 'end', 'outcome']

treatment = treatment.reindex(columns=new_order)

In [ ]:
treatment.head(40)

In [ ]:
exclude_col = 'outcome'
regimen = new_regimen_treatment.drop(columns=exclude_col)

In [ ]:
regimen.head(40)

In [ ]:
new_regimen = transform_regimen(regimen, registration)

In [ ]:
# Replacing the Adverse event value because the module function did not change it
# new_regimen['reason'] = new_regimen['reason'].str.replace('Adverse event', 'adverse')

In [ ]:
# I need to ask Urvi/Julia what to do with these NaN values in the reason column.
new_regimen.head(40)

#### Biochemistry Dataframe:

In [ ]:
biochem.head(40)

In [ ]:
rows_to_drop = [0]
biochemistry = drop_example_row(biochem, rows_to_drop)

# Drop the Action column 
biochemistry = biochemistry.iloc[:, :-1]

In [ ]:
# biochemistry['Date'] = biochemistry['Date'].replace('NO DATA', np.nan)

In [ ]:
new_biochemistry = transform_biochem(biochemistry)

In [ ]:
new_biochemistry.reset_index(drop=True, inplace=True)

In [ ]:
# TODO: I think I need to drop those NaN values in the result column
new_biochemistry.head(40)

#### For instances where Patient Case or Condition file is not in the XLSX country file.

In [ ]:
def save_patient_condition(dataframes, data_folder, country_month):
    index_labels = {1: 'patient', 2: 'condition'}
    timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M")
    
    # Loop through the dataframes list and save them 
    for idx, df in enumerate(dataframes, start=1):
        csv_file_name = f'{country_month}_{idx}_{timestamp}'
        
        # Conditionally add the label based on the index
        if idx in index_labels:
            csv_file_name += f'_{index_labels[idx]}'
        
        csv_file_name += '.csv'
        
        csv_file_path = os.path.join(data_folder, csv_file_name)
        df.to_csv(csv_file_path, index=False)

In [ ]:
processed_jan = pd.read_csv('../countries/Mali/data/processed/Mali_jan_2_2024-01-16_14-35.csv')

In [ ]:
processed_jan.head(20)

In [ ]:
patient_condition_jan = create_patient_condition_table(processed_jan, CaseBrowser_prod_data)

In [ ]:
# Print tuples of the dataframes
patient_condition_jan[0].head(10) # *patient table*

In [ ]:
patient_condition_jan[1].head(10) # *condition table*

In [ ]:
# Now I want to make the tuples into individual dataframes
Mali_jan_patient = patient_condition_jan[0]
Mali_jan_condition = patient_condition_jan[1]

In [ ]:
Mali_jan_condition.duplicated().sum()

In [ ]:
# drop_dups(Mali_jan_patient, 'identifier')
drop_dups(Mali_jan_condition, 'identifier')

In [ ]:
# List of dataframes to save
dataframes_patient_condition = [Mali_jan_patient, Mali_jan_condition]

save_patient_condition(dataframes_patient_condition, data_folder, country_month)

#### Save final dataframes into the processed data folder

In [ ]:
# Check for duplicates in the dataframes
check_dups(new_specimen, 'containeridentifier')

In [ ]:
# List of dataframes to save
dataframes = [patient, new_condition, new_microscopy, new_culture, new_specimen]

# Change the folder depending on the country 
data_folder = '../COUNTRIES/USA/data/processed/'
country_month = 'MLI_Dec'
timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M")

# Loop through the dataframes list and save them 
for idx, df in enumerate(dataframes, start=1):
    csv_file_path = os.path.join(data_folder, f'{country_month}_{idx}_{timestamp}.csv')
    df.to_csv(csv_file_path, index=False)